<a href="https://colab.research.google.com/github/zacharylazzara/tent-detection/blob/main/Tent_Detector_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:
* Fix CNN, it isn't generating any useful tent count
* Utilize spiking neural networks to implement UNet (try using Nengo?); we'll compare the effectiveness of the two approaches (maybe we should have multiple approaches, in which case we store the segmentor models in a dictionary we can iterate through?)

In [1]:
#@title Reset
RESET = False #@param {type:"boolean"}
if RESET:
  !if [[ "$PWD" = "/content" ]]; then rm -r ./*; fi

In [2]:
#@title Models
import os
from google.colab import files
UPLOAD_MODELS             = False        #@param {type:"boolean"}
FORCE_UPLOAD              = False       #@param {type:"boolean"}
SEGMENTOR_MODEL_FILENAME  = 'unet.pth'  #@param {type:"string"}
REGRESSOR_MODEL_FILENAME  = 'cnn.pth'   #@param {type:"string"}

uploaded_files = None
if UPLOAD_MODELS:
  if FORCE_UPLOAD:
    os.remove(SEGMENTOR_MODEL_FILENAME)
    os.remove(REGRESSOR_MODEL_FILENAME)
  if not (os.path.exists(SEGMENTOR_MODEL_FILENAME) and os.path.exists(REGRESSOR_MODEL_FILENAME)):
    uploaded_files = files.upload()
    for filename in uploaded_files.keys():
      print(f'Uploaded file "{filename}"')
  if not os.path.exists(SEGMENTOR_MODEL_FILENAME) or not os.path.exists(REGRESSOR_MODEL_FILENAME):
    raise Exception('Error: Missing one or more files!')

In [3]:
#@title Output
DOWNLOAD_OUTPUT           = True #@param {type:"boolean"}
KILL_RUNTIME_ON_COMPLETE  = True #@param {type:"boolean"}

In [4]:
#@title Config
TRAIN_MODELS  = True        #@param {type:"boolean"}
N_EPOCHS      = 200         #@param {type:"number"}
BATCH_SIZE    = 8           #@param {type:"number"}
INIT_LR       = 0.0001      #@param {type:"number"}
IMAGE_HEIGHT  = 512         #@param {type:"number"}
IMAGE_WIDTH   = 512         #@param {type:"number"}
TEST_SPLIT    = 0.15        #@param {type:"number"}
RANDOM_STATE  = 42          #@param {type:"number"}
OUTPUT_FORMAT = 'png'       #@param ["png", "jpg"] {allow-input: true}

#Initialization

In [5]:
#@title Imports
!pip install torchmetrics -q
import time
import math
import csv
import cv2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import runtime
from statistics import mean
from tqdm.auto import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch.nn import Sequential
from torch.nn import ModuleList
from torch.nn import ConvTranspose2d
from torch.nn import Flatten
from torch.nn import functional
from torch.nn import BatchNorm2d
from torch.nn import Softplus
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.nn.modules.loss import PoissonNLLLoss
from torchmetrics.classification import BinaryJaccardIndex
from torch import flatten
from torch import cat
from torch import randn
from torchvision import transforms
from torchvision.transforms import CenterCrop
from torchvision.utils import save_image
from torch.utils.data import Dataset
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD
import torchvision.transforms.functional as TF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 5.5 MB/s eta 0:00:00


In [6]:
#@title Device
DEVICE = None
if torch.cuda.is_available():
    DEVICE = 'cuda'
elif torch.backends.mps.is_available():
    DEVICE = 'mps'
else:
    DEVICE = 'cpu'
PIN_MEMORY = True if DEVICE != 'cpu' else False
print(f'Using device: {DEVICE}')

Using device: cuda


In [7]:
#@title Environment
%env SRC_DIR        = sarpol-zahab-tents
%env OUTPUT_DIR     = output

SRC_DIR             = os.environ.get('SRC_DIR')
OUTPUT_DIR          = os.environ.get('OUTPUT_DIR')

MODELS_PATH         = Path(f'{OUTPUT_DIR}/models')
METRICS_PATH        = Path(f'{OUTPUT_DIR}/metrics')
PREDS_PATH          = Path(f'{OUTPUT_DIR}/predictions')
OVERVIEW_PATH       = Path(f'{OUTPUT_DIR}/overviews')
OVERLAY_PATH        = Path(f'{OUTPUT_DIR}/overlays')

DATA_PATH           = Path(f'{SRC_DIR}/data')
IMAGES_PATH         = Path(f'{DATA_PATH}/images')
MASKS_PATH          = Path(f'{DATA_PATH}/labels')
LABELS_PATH         = Path(f'{DATA_PATH}/sarpol_counts.csv')
EXT                 = '.jpg'

env: SRC_DIR=sarpol-zahab-tents
env: OUTPUT_DIR=output


In [8]:
#@title Directories
%%bash

if [ -d 'sample_data' ]; then
  rm -r sample_data
fi

if [ ! -d $SRC_DIR ]; then
  git clone https://github.com/tofighi/sarpol-zahab-tents.git
fi

if [ ! -d $OUTPUT_DIR ]; then
  mkdir -p $OUTPUT_DIR/models
  mkdir -p $OUTPUT_DIR/metrics
  mkdir -p $OUTPUT_DIR/predictions
  mkdir -p $OUTPUT_DIR/overviews
  mkdir -p $OUTPUT_DIR/overlays
fi

Cloning into 'sarpol-zahab-tents'...


#Models

In [9]:
#@title UNet
# Adapted from: https://pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/

class Block(Module):
  def __init__(self, in_channels, out_channels):
    super(Block, self).__init__()
    self.double_conv2d = Sequential(
        Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
        BatchNorm2d(out_channels),
        ReLU(inplace=True),
        Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        BatchNorm2d(out_channels),
        ReLU(inplace=True)
    )

  def forward(self, x):
    return self.double_conv2d(x)

class Encoder(Module):
  def __init__(self, channels=(3, 16, 32, 64)):
    super(Encoder, self).__init__()
    self.encoder_blocks = ModuleList([Block(channels[i], channels[i+1]) for i in range(len(channels)-1)])
    self.pool = MaxPool2d(2)

  def forward(self, x):
    block_outputs = []
    for block in self.encoder_blocks:
      x = block(x)
      block_outputs.append(x)
      x = self.pool(x)
    return block_outputs

class Decoder(Module):
  def __init__(self, channels=(64, 32, 16)):
    super(Decoder, self).__init__()
    self.up_convs = ModuleList([ConvTranspose2d(channels[i], channels[i+1], 2, 2) for i in range(len(channels)-1)])
    self.decoder_blocks = ModuleList([Block(channels[i], channels[i+1]) for i in range(len(channels)-1)])
  
  def crop(self, encoder_features, x):
    (_, _, H, W) = x.shape
    return CenterCrop([H, W])(encoder_features)
  
  def forward(self, x, encoder_features):
    for i in range(len(self.up_convs)):
      x = self.up_convs[i](x)
      encoder_feature = self.crop(encoder_features[i], x)
      x = cat([x, encoder_feature], dim=1)
      x = self.decoder_blocks[i](x)
    return x

class UNet(Module):
  def __str__(self) -> str:
    return 'UNet'

  def __init__(self, encoder_channels=(3, 16, 32, 64), decoder_channels=(64, 32, 16), classes=1, retain_dim=True, output_size=(512, 512)):
    super(UNet, self).__init__()
    self.encoder = Encoder(encoder_channels)
    self.decoder = Decoder(decoder_channels)
    self.head = Conv2d(decoder_channels[-1], classes, 1)
    self.retain_dim = retain_dim
    self.output_size = output_size

  def forward(self, x):
    encoder_features = self.encoder(x)
    decoder_features = self.decoder(encoder_features[::-1][0], encoder_features[::-1][1:])
    map = self.head(decoder_features)
    if self.retain_dim:
      map = functional.interpolate(map, self.output_size)
    return map

In [10]:
#@title CNN
class CNN(Module):
  def __str__(self) -> str:
    return 'CNN'

  def __init__(self):
    super(CNN, self).__init__()
    self.model = Sequential(
        Conv2d(1, 64, 3),
        ReLU(),
        MaxPool2d(2),
        Conv2d(64, 64, 3),
        ReLU(),
        MaxPool2d(2),
        Flatten(),
        Linear(1016064, 1),
        ReLU(),
        Linear(1, 1)
    )

  def forward(self, x):
    return self.model(x).t().squeeze()

In [11]:
#@title Dataset
# Adapted from: https://pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/
class SegmentationDataset(Dataset):
  def __init__(self, dataframe, transformations = None):
    self.dataframe = dataframe
    self.transformations = transformations

  def __len__(self):
    return len(self.dataframe.index)

  def __getitem__(self, index):
    image = cv2.cvtColor(cv2.imread(self.dataframe.iloc[index]['image_paths']), cv2.COLOR_BGR2RGB)
    mask = cv2.threshold(cv2.imread(self.dataframe.iloc[index]['mask_paths'], cv2.IMREAD_GRAYSCALE), 150, 255, cv2.THRESH_BINARY)[1]
    if self.transformations is not None:
      image = self.transformations(image)
      mask = self.transformations(mask)
    return (image, mask, self.dataframe.iloc[index]['labels'], self.dataframe.index[index])

#Functions

In [12]:
#@title Load
def load_data(images_path=IMAGES_PATH, masks_path=MASKS_PATH, csv_path=LABELS_PATH):
  with open(csv_path) as csv_file:
    rows = [row for row in csv.reader(csv_file)]
    return pd.DataFrame({
        'names'        : [row[0].split('.')[0] for row in rows],
        'image_paths'  : [str(next(images_path.glob(row[0]))) for row in rows],
        'mask_paths'   : [str(next(masks_path.glob(row[0]))) for row in rows],
        'labels'       : [int(row[1]) for row in rows]
    }).set_index('names')

In [13]:
#@title Train
# Adapted from: https://pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/
def train(model, t_loader, v_loader, loss_func, opt, metric=None, epochs=N_EPOCHS):
  history = pd.DataFrame({
      't': {'losses':[], 'metrics':[]},
      'v': {'losses':[], 'metrics':[]}
  })

  if metric != None:
    metric.to(DEVICE)

  progress_bar = tqdm(range(epochs))
  for e in progress_bar:
    model.train()
    losses = []
    for (i, (x, y, c, _)) in enumerate(t_loader):
      (x, y, c) = (x.to(DEVICE), y.to(DEVICE), c.to(DEVICE))
      feature = (y if str(model) == 'CNN' else x)
      target = (c if str(model) == 'CNN' else y)

      pred = model(feature)
      loss = loss_func(pred, target)
      losses.append(loss.item())

      if loss.requires_grad:
        opt.zero_grad()
        loss.backward()
      opt.step()

      if metric != None:
        metric.update(pred, target)
        
    history['t']['losses'].append(mean(losses))
    if metric != None:
      history['t']['metrics'].append(metric.compute().cpu().detach().numpy())
      metric.reset()
    
    with torch.no_grad():
      model.eval()
      
      losses = []
      for (x, y, c, _) in v_loader:
        (x, y, c) = (x.to(DEVICE), y.to(DEVICE), c.to(DEVICE))
        feature = (y if str(model) == 'CNN' else x)
        target = (c if str(model) == 'CNN' else y)

        pred = model(feature)
        loss = loss_func(pred, target)
        losses.append(loss.item())

        if metric != None:
          metric.update(pred, target)
      
      history['v']['losses'].append(mean(losses))
      if metric != None:
        history['v']['metrics'].append(metric.compute().cpu().detach().numpy())
        metric.reset()

    progress_bar.set_description(f'Epoch({e+1}/{N_EPOCHS}) Training {model}, Train Loss: {history["t"]["losses"][-1]:.4f}, Test Loss: {history["v"]["losses"][-1]:.4f}')
  return history

In [14]:
#@title Predict
def predict(model, loader, output_dir=PREDS_PATH):
  preds = []
  with torch.no_grad():
    model.eval()
    progress_bar = tqdm(loader)
    progress_bar.set_description(f'Evaluating {model}')
    for (x, y, _, name) in progress_bar:
      (x, y) = (x.to(DEVICE), y.to(DEVICE))
      feature = (y if str(model) == 'CNN' else x)
      pred = model(feature)
      if str(model) == 'CNN':
        for batch, c in enumerate(pred.cpu().detach()):
          preds.append({'names':name[batch], 'image_paths':str(next(IMAGES_PATH.glob(f'{name[batch]}{EXT}'))), 'mask_paths':None, 'labels':c.numpy()})
          progress_bar.set_description(f'Predicting label for {name[batch]}')
      else:
        for batch, img in enumerate(pred.cpu().detach()):
          out_path = f'{output_dir}/{name[batch]}.{OUTPUT_FORMAT}'
          save_image(img, out_path)
          preds.append({'names':name[batch], 'image_paths':str(next(IMAGES_PATH.glob(f'{name[batch]}{EXT}'))), 'mask_paths':out_path, 'labels':None})
          progress_bar.set_description(f'Saved prediction for {name[batch]} to {out_path}')
  return pd.DataFrame(preds).set_index('names').fillna(np.nan)

In [15]:
#@title Tile
def tile(loader, output_path_x, output_path_y):
  output_x = output_y = None
  progress_bar = tqdm(loader)
  progress_bar.set_description('Tiling')
  for (x, y, _, _) in progress_bar:
    if output_x == None and output_y == None:
      output_x = torch.cat(tuple(x), 2)
      output_y = torch.cat(tuple(y), 2)
    else:
      output_x = torch.cat((output_x, torch.cat(tuple(x), 2)), 1)
      output_y = torch.cat((output_y, torch.cat(tuple(y), 2)), 1)

  print('Saving...')
  save_image(output_x, output_path_x)
  save_image(output_y, output_path_y)
  print('Done.')

In [16]:
#@title Overlay

# TODO: adapt this from the 2022 version
# def overlay(image_path, mask_path, output_path=OVERLAY_PATH):
#   img = None
#   msk = None
#   with Image.open(image_path).convert('RGBA') as image:
#     img = np.array(image)
#   with Image.open(mask_path).convert('RGBA') as mask:
#     msk = np.array(mask)

#   for channel in range(1, 2):
#     msk[msk[:,:,channel] > 0, channel] = 0

#   overlay = cv2.addWeighted(img, 1, msk, 1, 0)

#   output = Image.fromarray(overlay)
#   output.save(f'{output_path}', format=f'{OUTPUT_FORMAT}')

#Main

##Data

In [17]:
#@title Training and Validation Loaders
transformations = transforms.Compose([transforms.ToPILImage(), transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)), transforms.ToTensor()])

training_data, validation_data = train_test_split(load_data(), test_size=TEST_SPLIT, random_state=RANDOM_STATE)
training_dataset = SegmentationDataset(training_data, transformations)
validation_dataset = SegmentationDataset(validation_data, transformations)

t_loader = DataLoader(training_dataset, shuffle=True, batch_size=BATCH_SIZE, pin_memory=PIN_MEMORY, num_workers=os.cpu_count(), persistent_workers=True)
v_loader = DataLoader(validation_dataset, shuffle=True, batch_size=BATCH_SIZE, pin_memory=PIN_MEMORY, num_workers=os.cpu_count(), persistent_workers=True)

In [18]:
#@title Split Ratio
t_count = len(training_data)
v_count = len(validation_data)

t_ratio = 1-v_count/t_count
v_ratio = v_count/t_count

print(f'Training to Validation Ratio\n')
print(f'Training ({t_count}): \t{t_ratio*100:>10.2f}%')
print(f'Validation ({v_count}): \t{v_ratio*100:>10.2f}%')
print(f'Total ({t_count + v_count}): \t\t{t_ratio*100 + v_ratio*100:>10.2f}%') 

assert t_ratio + v_ratio == 1 # Sanity Check

Training to Validation Ratio

Training (217): 	     82.03%
Validation (39): 	     17.97%
Total (256): 		    100.00%


## Model Operations

In [19]:
#@title Load Models
segmentor = regressor = None
if UPLOAD_MODELS:
  segmentor = torch.load(SEGMENTOR_MODEL_FILENAME, map_location=DEVICE).to(DEVICE)
  regressor = torch.load(REGRESSOR_MODEL_FILENAME, map_location=DEVICE).to(DEVICE)
else:
  segmentor = UNet().to(DEVICE)
  regressor = CNN().to(DEVICE)

In [20]:
#@title Train Models
segmentor_results = None
regressor_results = None
if TRAIN_MODELS:
  segmentor_results = train(segmentor, t_loader, v_loader, BCEWithLogitsLoss(), Adam(segmentor.parameters(), lr=INIT_LR), BinaryJaccardIndex())
  regressor_results = train(regressor, t_loader, v_loader, PoissonNLLLoss(), Adam(regressor.parameters(), lr=INIT_LR))
  torch.save(segmentor, f'{MODELS_PATH}/{SEGMENTOR_MODEL_FILENAME}')
  torch.save(regressor, f'{MODELS_PATH}/{REGRESSOR_MODEL_FILENAME}')

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [21]:
#@title Metrics
if TRAIN_MODELS:
  #Segmentor Loss
  plt.plot(segmentor_results['t']['losses'], label='train loss')
  plt.plot(segmentor_results['v']['losses'], label='test loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.ylim([0, 1])
  plt.legend(loc='lower right')
  plt.title('UNet')
  plt.savefig(f'{METRICS_PATH}/unet_loss.{OUTPUT_FORMAT}')
  plt.close()

  #Segmentor Metrics
  plt.plot(segmentor_results['t']['metrics'], label='train metrics')
  plt.plot(segmentor_results['v']['metrics'], label='test metrics')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.ylim([0, 1])
  plt.legend(loc='lower right')
  plt.title('UNet')
  plt.savefig(f'{METRICS_PATH}/unet_metrics.{OUTPUT_FORMAT}')
  plt.close()

  #Regressor Loss
  plt.plot(regressor_results['t']['losses'], label='train loss')
  plt.plot(regressor_results['v']['losses'], label='test loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  # plt.ylim([0, 1])
  plt.legend(loc='lower right')
  plt.title('CNN')
  plt.savefig(f'{METRICS_PATH}/cnn_loss.{OUTPUT_FORMAT}')
  plt.close()

##Output

In [22]:
#@title Prediction Loaders
transformations = transforms.Compose([transforms.ToPILImage(), transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)), transforms.ToTensor()])
dataset = SegmentationDataset(load_data(), transformations)
row_size = int(math.sqrt(len(dataset))) # Batch of 16 since each row is 16 images long
loader = DataLoader(dataset, shuffle=False, batch_size=row_size, pin_memory=PIN_MEMORY, num_workers=os.cpu_count())

#Segmentor
segmentor_predictions = predict(segmentor, loader)
segmentor_dataset = SegmentationDataset(segmentor_predictions, transformations)
segmentor_loader = DataLoader(segmentor_dataset, shuffle=False, batch_size=row_size, pin_memory=PIN_MEMORY, num_workers=os.cpu_count(), persistent_workers=True)

#Regressor
regressor_predictions = predict(regressor, segmentor_loader)
predictions = segmentor_predictions.combine_first(regressor_predictions).sort_index()
predictions.to_csv(f'{PREDS_PATH}/labels.csv')
prediction_dataset = SegmentationDataset(predictions, transformations)
prediction_loader = DataLoader(prediction_dataset, shuffle=False, batch_size=row_size, pin_memory=PIN_MEMORY, num_workers=os.cpu_count())

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [23]:
#@title Save Overview

# TODO: we need to adjust tile to take only one value from the loader, and then pass it a subset of the prediction loader

# tile(loader, f'{OVERVIEW_PATH}/x_overview.{OUTPUT_FORMAT}', f'{OVERVIEW_PATH}/y_overview.{OUTPUT_FORMAT}')
# TODO: make tile ignore x if we provide no x path (we want to tile on both x, y, and the prediction)

tile(prediction_loader, f'{OVERVIEW_PATH}/x_overview.{OUTPUT_FORMAT}', f'{OVERVIEW_PATH}/p_overview.{OUTPUT_FORMAT}')

  0%|          | 0/16 [00:00<?, ?it/s]

Saving...
Done.


In [24]:
#@title Zip and Download Output
if DOWNLOAD_OUTPUT:
  !zip -rq tent_detector_output.zip $OUTPUT_DIR
  files.download('tent_detector_output.zip')

  # TODO: find a way to wait for the files to finish downloading so we can actually end the session once we're done...

  # if KILL_RUNTIME_ON_COMPLETE:
  #   time.sleep(60*10) # Because I can't find any mention of if file.download can be run synchronously.
  #   print('Session Terminated on Complete.')
  #   runtime.unassign()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>